In [1]:
import os, sys
import urllib
import zipfile
import gzip
import pandas as pd

from collections import defaultdict
from goatools.anno.gaf_reader import GafReader
from goatools.obo_parser import GODag

import networkx as nx

data_dir = "../data/"

/tmp/ipykernel_38418/2207716882.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def download_and_unzip(download_url_link, dir_path, zipped_filename,destination_dir_name, unzip=True):
    #https://www.tutorialsbuddy.com/download-and-unzip-a-zipped-file-in-python
    print("Download starting")

    urllib.request.urlretrieve(
        download_url_link, os.path.join(dir_path, zipped_filename)
    )
    print("Download complete")

    if unzip:
        print("unzipping file starting")
    
        if zipped_filename.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(dir_path, zipped_filename), "r") as zip_file:
                zip_file.extractall(os.path.join(dir_path, destination_dir_name))
        elif zipped_filename.endswith(".gz"):
            print("zipfile")
            with gzip.GzipFile(os.path.join(dir_path, zipped_filename), "rb") as zip_file:
                with open(os.path.join(dir_path, destination_dir_name, zipped_filename.replace(".gz", "")), "wb") as fout:
                    fout.write(zip_file.read())
        else:
            raise NotImplementedError("NO CASE")
            
    
    print("unzipping complete")

In [3]:
if not os.path.exists(os.path.join(data_dir, "goa_human.gaf")):
    download_and_unzip("http://geneontology.org/gene-associations/goa_human.gaf.gz", ".", os.path.join(data_dir, "goa_human.gaf.gz"), ".")
        

In [4]:
if not os.path.exists(os.path.join(data_dir, "go-basic.obo")):
    download_and_unzip("http://geneontology.org/ontology/go-basic.obo", ".", os.path.join(data_dir, "go-basic.obo"), ".", unzip=False)
        

In [5]:
genenamesURL = 'https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=gd_status&col=gd_pub_acc_ids&col=gd_pub_refseq_ids&col=md_prot_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit'

if not os.path.exists(os.path.join(data_dir, "hgnc_annot.tsv")):
    download_and_unzip(genenamesURL, ".", os.path.join(data_dir, "hgnc_annot.tsv"), ".", unzip=False)

In [6]:
ogaf = GafReader(os.path.join(data_dir, "goa_human.gaf"))
obodag = GODag(os.path.join(data_dir, "go-basic.obo"))

HMS:0:00:10.758043 626,136 annotations READ: ../data/goa_human.gaf 
../data/go-basic.obo: fmt(1.2) rel(2023-01-01) 46,739 Terms


In [7]:
hgncDF = pd.read_csv(os.path.join(data_dir, "hgnc_annot.tsv"), sep="\t")
uniprot2hgnc = defaultdict(set)
all_genes = set()

for ri, row in hgncDF.iterrows():
    
    status = row["Status"]
    
    if status == "Symbol Withdrawn":
        continue
    
    symbol = row["Approved symbol"]    
    uniprot= row["UniProt ID(supplied by UniProt)"]
    
    all_genes.add(symbol)
    
    if pd.isna(uniprot):
        continue

    uniprot2hgnc[uniprot].add(symbol)
        

In [8]:
len(all_genes)

45406

In [9]:
go2gene = defaultdict(set)

for assoc in ogaf.get_associations():

    geneID = assoc.DB_ID
    goID = assoc.GO_ID
    
    if not geneID in uniprot2hgnc:
        geneID = assoc.DB_Symbol
        geneIDs = [geneID]
        
        if not geneID in all_genes:
            print(geneID, assoc.Taxon, assoc)
            continue
    else:
        geneIDs = uniprot2hgnc[geneID]
    
    for geneSym in geneIDs:
        go2gene[goID].add((geneSym, tuple(assoc.Qualifier)))


LOC107984156 [9606] ntgafobj(DB='UniProtKB', DB_ID='A0A0G2JMH3', DB_Symbol='LOC107984156', Qualifier={'enables'}, GO_ID='GO:0003924', DB_Reference={'GO_REF:0000002'}, Evidence_Code='IEA', With_From={'InterPro:IPR006689'}, NS='MF', DB_Name={'ARL17 domain-containing protein'}, DB_Synonym={'LOC107984156'}, DB_Type='protein', Taxon=[9606], Date=datetime.date(2022, 11, 9), Assigned_By='InterPro', Extension=None, Gene_Product_Form_ID=set())
A0A0G2JMS6 [9606] ntgafobj(DB='UniProtKB', DB_ID='A0A0G2JMS6', DB_Symbol='A0A0G2JMS6', Qualifier={'enables'}, GO_ID='GO:0004867', DB_Reference={'GO_REF:0000043'}, Evidence_Code='IEA', With_From={'UniProtKB-KW:KW-0722'}, NS='MF', DB_Name={'Ovostatin'}, DB_Synonym=set(), DB_Type='protein', Taxon=[9606], Date=datetime.date(2022, 11, 9), Assigned_By='UniProt', Extension=None, Gene_Product_Form_ID=set())
A0A0G2JMS6 [9606] ntgafobj(DB='UniProtKB', DB_ID='A0A0G2JMS6', DB_Symbol='A0A0G2JMS6', Qualifier={'involved_in'}, GO_ID='GO:0010466', DB_Reference={'GO_REF:00

In [10]:
g = nx.DiGraph()

In [11]:
for gene in all_genes:
    g.add_node(gene, type="gene", score=0)

In [19]:
obodag["GO:0006915"]

GOTerm('GO:0006915'):
  id:GO:0006915
  item_id:GO:0006915
  name:apoptotic process
  namespace:biological_process
  _parents: 1 items
    GO:0012501
  parents: 1 items
    GO:0012501	level-03	depth-03	programmed cell death [biological_process]
  children: 18 items
  level:4
  depth:4
  is_obsolete:False
  alt_ids: 2 items
    GO:0006917
    GO:0008632

In [12]:
#add all nodes with attributes
for goEntry in obodag:
    
    termID = goEntry
    termObj = obodag[goEntry]
    
    if termObj.is_obsolete:
        continue
    
    termName = termObj.name
    termNS = termObj.namespace
    
    g.add_node(termID, id=termID, name=termName, ns=termNS, score=0)
    
for goEntry in obodag:
    
    termID = goEntry
    termObj = obodag[goEntry]
    
    for child in termObj.children:
        if child in g.nodes and not (child, goEntry) in g.edges:
            g.add_edge(child, goEntry, type="part_of", score=0)
            
    for parent in termObj.parents:
        if parent.id in g.nodes and not (goEntry, parent.id) in g.edges:
            g.add_edge(goEntry, parent.id, type="part_of", score=0)

In [13]:
print(g)

DiGraph with 92145 nodes and 74934 edges


In [14]:
[x for x in g.nodes][:5]

['SBK2', 'LIM2-AS1', 'ROPN1L', 'LINC01341', 'MIR518E']

In [15]:
all_interactions = set()
for goID in list(go2gene):   
    for gene, interaction in go2gene[goID]:
        for x in interaction:
            all_interactions.add(x)
all_interactions

{'NOT',
 'acts_upstream_of',
 'acts_upstream_of_negative_effect',
 'acts_upstream_of_or_within',
 'acts_upstream_of_or_within_negative_effect',
 'acts_upstream_of_or_within_positive_effect',
 'acts_upstream_of_positive_effect',
 'colocalizes_with',
 'contributes_to',
 'enables',
 'involved_in',
 'is_active_in',
 'located_in',
 'part_of'}

In [16]:
interaction_harmonize = {
    'NOT': "interacts",
    'acts_upstream_of': "interacts",
    'acts_upstream_of_negative_effect': "interacts",
    'acts_upstream_of_or_within': "interacts",
    'acts_upstream_of_or_within_negative_effect': "interacts",
    'acts_upstream_of_or_within_positive_effect': "interacts",
    'acts_upstream_of_positive_effect': "interacts",
    'colocalizes_with': "interacts",
    'contributes_to': "activates",
    'enables': "activates",
    'involved_in': "activates",
    'is_active_in': "activates",
    'located_in': "interacts",
    'part_of': "interacts"
    }

In [17]:
for goID in list(go2gene)[:5]:
    print(goID, go2gene[goID])
    
    for gene, interaction in go2gene[goID]:
        g.add_edge( gene, goID, type=interaction_harmonize[interaction[0]], go_interaction = interaction[0])

GO:0003723 {('SETX', ('enables',)), ('USP17L27', ('enables',)), ('RSRC2', ('enables',)), ('THUMPD1', ('enables',)), ('IMPDH1', ('enables',)), ('NOLC1', ('enables',)), ('H1-5', ('enables',)), ('TIA1', ('enables',)), ('NVL', ('enables',)), ('SCAF1', ('enables',)), ('RPL30', ('enables',)), ('SUPT16H', ('enables',)), ('APOBEC3A', ('enables',)), ('EIF1', ('enables',)), ('SURF6', ('enables',)), ('KYAT3', ('enables',)), ('DDX20', ('enables',)), ('RPS15A', ('enables',)), ('TES', ('enables',)), ('TFRC', ('enables',)), ('UTP23', ('enables',)), ('EIF3CL', ('enables',)), ('MRPS9', ('enables',)), ('FKBP3', ('enables',)), ('SERPINH1', ('enables',)), ('NXF2', ('enables',)), ('FUBP1', ('enables',)), ('EIF5AL1', ('enables',)), ('TSN', ('enables',)), ('HSPB1', ('enables',)), ('YTHDF1', ('enables',)), ('PPP1R8', ('enables',)), ('YWHAG', ('enables',)), ('SECISBP2L', ('enables',)), ('RPS5', ('enables',)), ('SRPK2', ('enables',)), ('CNOT1', ('enables',)), ('NR0B1', ('enables',)), ('ZNFX1', ('enables',)), ('

In [18]:
print(g)

DiGraph with 92145 nodes and 89524 edges
